In [0]:
%sql
USE CATALOG hive_metastore;
USE SCHEMA default;

In [0]:
%sql
DROP TABLE IF EXISTS CIRCUITS_EXT; 
CREATE TABLE CIRCUITS_EXT (
  circuitRef STRING,
  name STRING,
  location STRING,
  country STRING,
  lat DOUBLE,
  lng DOUBLE,
  alt INT,
  url STRING
)
USING csv
OPTIONS (path "/mnt/dbxdl/demo/circuits.csv", header "true");

In [0]:
%sql
SELECT * FROM CIRCUITS_EXT;

In [0]:
%sql
DESC DATABASE EXTENDED default;

In [0]:
%sql
DESCRIBE TABLE EXTENDED hive_metastore.default.CIRCUITS_EXT;

The table is **EXTERNAL** + **CSV** (see “Provider: csv”). Spark/Databricks doesn’t support UPDATE/DELETE/MERGE on CSV

## Stored in Hive_metastore

In [0]:
%python
df = spark.read \
  .option('inferSchema', True) \
  .option('header', True) \
  .csv('/mnt/dbxdl/demo/circuits.csv')

display(df)

In [0]:
%python
# Writing this dataframe to Delta
df.write.format('delta').mode('overwrite').saveAsTable('CIRCUITS_MGN_DELTA_HIVE_META')

In [0]:
%sql
SELECT * FROM CIRCUITS_MGN_DELTA_HIVE_META LIMIT 5;

In [0]:
%sql
DESCRIBE TABLE EXTENDED hive_metastore.default.CIRCUITS_MGN_DELTA_HIVE_META;

In [0]:
%python
#display(dbutils.fs.mounts())
display(dbutils.fs.ls("dbfs:/user/hive/warehouse/circuits_mgn_delta_hive_meta"))

## Hive_metastore
### Conclution
**Anything you create in the default hive_metastore—including Delta files—is tied to that Databricks workspace and isn’t fully under your control; to retain ownership and manageability over your data, store your Delta files in an external location instead.**

## Write to a different Container - Storage Account

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS DELTA_DB
LOCATION '/mnt/dbxdl/delta';

In [0]:
%sql
USE DATABASE DELTA_DB;

In [0]:
%python
df = spark.read \
  .option('inferSchema', True) \
  .option('header', True) \
  .csv('/mnt/dbxdl/demo/circuits.csv')

display(df)

In [0]:
%python
# Writing this dataframe to Delta
df.write.format('delta').mode('overwrite').saveAsTable('CIRCUITS_MGN_DELTA')

In [0]:
%sql
SELECT * FROM CIRCUITS_MGN_DELTA;

In [0]:
%sql
DESC TABLE EXTENDED CIRCUITS_MGN_DELTA;

In [0]:
%sql
DROP TABLE IF EXISTS CIRCUITS_MGN_DELTA;